In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
import requests
import os
import time
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from threading import Thread, Lock
import threading
import pickle
# import undetected_chromedriver.v2 as uc
import datetime


### РИА новости

In [17]:
# Сохраняем всю ленту новостей

def get_source_html(url, driver):
    counter = 0
    driver.get(url)

    time.sleep(30)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    action = ActionChains(driver)
    el = driver.find_element(By.CLASS_NAME, 'list-more')
    action.move_to_element_with_offset(el, 5, 5)
    action.click()
    action.perform()

    time.sleep(0.5)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    try:
        counter += 2

        while True:

            if counter >= 300 and counter % 20 == 0:
                with  open(f'ria_economy/raw_preview/news_ria{counter}.html', 'w') as file:
                    file.write(driver.page_source)

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)
            counter += 1

    except Exception as _ex:
        print(_ex)

    finally:
        driver.close()
        driver.quit()

In [21]:
# Сохраняем ссылки из ленты новостей

def get_file_urls(file_path):
    import os
    files = sorted(os.listdir(file_path))[1:]

    urls = []
    c = 1
    for f in files:

        with open(file_path + f'/{f}') as file:
            src = file.read()
        
        soup = BeautifulSoup(src, 'lxml')
        items_divs = soup.find_all('div', class_='list-item')

        for item in items_divs:
            item_url = item.find('div', class_='list-item__content').find('a').get('href')
            if item_url not in urls:
                urls.append(item_url)

        print(f'Ссылки из файлов {c} из {len(files)} получены')
        c+=1

    with open(f'ria_economy/urls.txt', 'w') as file:
        for url in urls:
            file.write(f'{url}\n')

        print(f'Ссылки сохранены')
    return

In [20]:
# Запускаем функцию для сбора ссылок, останавливаем вручную, потому что нельзя листать ее до бесконечности.

driver = webdriver.Chrome(ChromeDriverManager().install())

get_source_html(url='https://ria.ru/politics/', driver=driver)

/var/folders/m6/2n44mlkj7nbdwb651dd78pwc0000gn/T/ipykernel_810/1943467916.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


KeyboardInterrupt: 

In [37]:
# Вытаскиваем ссылки
file_path = '/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_politics/raw_preview'
get_file_urls(file_path)

Ссылки из файлов 1 из 23 получены
Ссылки из файлов 2 из 23 получены
Ссылки из файлов 3 из 23 получены
Ссылки из файлов 4 из 23 получены
Ссылки из файлов 5 из 23 получены
Ссылки из файлов 6 из 23 получены
Ссылки из файлов 7 из 23 получены
Ссылки из файлов 8 из 23 получены
Ссылки из файлов 9 из 23 получены
Ссылки из файлов 10 из 23 получены
Ссылки из файлов 11 из 23 получены
Ссылки из файлов 12 из 23 получены
Ссылки из файлов 13 из 23 получены
Ссылки из файлов 14 из 23 получены
Ссылки из файлов 15 из 23 получены
Ссылки из файлов 16 из 23 получены
Ссылки из файлов 17 из 23 получены
Ссылки из файлов 18 из 23 получены
Ссылки из файлов 19 из 23 получены
Ссылки из файлов 20 из 23 получены
Ссылки из файлов 21 из 23 получены
Ссылки из файлов 22 из 23 получены
Ссылки из файлов 23 из 23 получены
Ссылки сохранены


In [2]:
# Загружаем сохраненные ссылки
with open('ria_politics/urls.txt') as file:
    urls = file.read()
urls = urls.split('\n')

In [2]:
# Создаем набор user agent, чтобы сайт не так быстро блочил
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

In [3]:
# Создаем массив данных, куда будут все склвдывать потоки
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

NameError: name 'urls' is not defined

In [4]:
# Пишем функцию для парсинга нужных данных с многопоточностью.

def parse_from_link(link, user_agent, current_value):
    if current_value % 10 == 0:
        user_agent = user_agent_rotator.get_random_user_agent()

    try:
        with requests.session() as s:
            txt = s.get(link, headers={'User-Agent':user_agent}).text
            soup = BeautifulSoup(txt, 'lxml')
            try:
                timestamp = soup.find('div', class_='article__info-date').find('a').text
            except:
                timestamp = 'No time'
            
            try:
                title = soup.find('h1', class_='article__title').text
            except:
                title = 'No title'
            try:
                announce = soup.find('div', class_='article__announce-text').text
            except:
                announce = 'No announce'
            try:
                text = soup.find('div', class_='article__body js-mediator-article mia-analytics').text
            except:
                text = 'No text'
            return timestamp, title, announce, text

    except Exception as ex:
        print(link)
        print(ex)
        return ex

    except KeyboardInterrupt:
        return


def parse(lock, links, save_evry):

    global data, total_news, current_value

    user_agent = user_agent_rotator.get_random_user_agent()

    while current_value < total_news:
        with lock:
            thread_value = current_value
            link = links[current_value]
            current_value += 1
        
        parsed = parse_from_link(link, user_agent, thread_value)
        if parsed == None:
            return

        with lock:
            data['link'].append(link)
            data['data_or_ex'].append(parsed)
            if thread_value % save_evry == 0:
                with open(f'ria_economy/raw_cites/{thread_value}_skipped.pkl', 'wb') as f:
                    pickle.dump(data, f)
                    print(f'{thread_value} saved')
                data.clear()
                data['link'] = []
                data['data_or_ex'] = []

        time.sleep(2)

    with open(f'ria_economy/raw_cites/{thread_value}_skipped.pkl', 'wb') as f:
        pickle.dump(data, f)

    return 'Конец'

def start_parsing(links, n_threads=3, save_evry=15000):
    
    lock = Lock()
    threads = []
    run_event = threading.Event()
    run_event.set()

    for i in range(n_threads):
        t = Thread(target=parse, args=(lock, links, save_evry))
        t.start()
        time.sleep(2)
        threads.append(t)

    for thread in threads:
        thread.join()

In [ ]:
# Запускаем функцию
start_parsing(urls, n_threads=2, save_evry=200)

### Дальше по сути тот же код, что и был выше, но я там докачивал пропуски

In [13]:
with open('ria_politics/urls_retry.txt') as file:
    urls = file.read()
urls = urls.split('\n')

data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [14]:
start_parsing(urls, n_threads=2, save_evry=200)

0 saved
200 saved
400 saved

Invalid URL '': No scheme supplied. Perhaps you meant http://?


In [6]:
with open('ria_economy/urls_retry.txt') as file:
    urls = file.read()
urls = urls.split('\n')

data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [7]:
start_parsing(urls, n_threads=2, save_evry=200)

0 saved
200 saved
400 saved
600 saved
800 saved
1000 saved
1200 saved
1400 saved
1600 saved
1800 saved
2000 saved
2200 saved
2400 saved
2600 saved
2800 saved
3000 saved

Invalid URL '': No scheme supplied. Perhaps you meant http://?


In [ ]:
with open('ria_politics/urls_retry.txt') as file:
    urls = file.read()
urls = urls.split('\n')

In [24]:
def urls_left(file_path):
    urls = []
    files = sorted(os.listdir(f'{file_path}'))
    
    for file in files:
        try:
            df = pd.DataFrame(pd.read_pickle(f'{file_path}/{file}'))
            df['tp'] = df['data_or_ex'].map(lambda x: isinstance(x, tuple))
            df.drop(df[df.tp == True].index, inplace=True)
            df = df[df['link'].str.contains('https://ria.ru/')==True]
            url = df['link'].astype(str).to_list()
            urls.extend(url)
        except Exception as ex:
            print(ex)
    return urls

In [21]:
politics_path = '/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_politics/raw_cites'
economics_path = '/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_economy/raw_cites'

In [25]:
urls_politics = urls_left(politics_path)
urls_economics = urls_left(economics_path)

All arrays must be of the same length
All arrays must be of the same length
invalid load key, '\x00'.
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!
Can only use .str accessor with string values!


In [26]:
with open(f'ria_politics/urls_retry.txt', 'w') as file:
    for url in urls_politics:
        file.write(f'{url}\n')

with open(f'ria_economy/urls_retry.txt', 'w') as file:
    for url in urls_economics:
        file.write(f'{url}\n')

In [2]:
pd.read_csv('ria_politics/prep_cites/200.csv')

,link,date,title,announce,text
0,https://ria.ru/20110211/333378123.html,19:57 11.02.2011,активист хабаровск приносить консульство япони...,активист молодежный движение хабаровск приноси...,NaN
1,https://ria.ru/20110211/333385555.html,20:13 11.02.2011,токио протестовать против медведев снова поеха...,выражать серьезный протест отношение визит сос...,москва 11 фев риа новость япония категорически...
2,https://ria.ru/20110211/333377675.html,19:56 11.02.2011,иран назло весь враг,иран отмечать 32 годовщина исламский революция...,иран отмечать 32 годовщина исламский революция...
3,https://ria.ru/20110211/333366199.html,19:26 11.02.2011,российский нация должный состоять самобытный н...,большой задача касаться формирование будущий и...,уфа 11 фев риа новость президент рф дмитрий ме...
4,https://ria.ru/20110211/333358165.html,19:05 11.02.2011,медведев называть межнациональный единство пер...,межнациональный единство мочь государство оно ...,уфа 11 фев риа новость руководитель весь регио...
...,...,...,...,...,...
194,https://ria.ru/20110203/329935025.html,17:32 03.02.2011,рф подписывать соглашение ущерб свой безопасно...,разумеется договор сокращение наступательный в...,москва 3 фев риа новость весь решение сфера бе...
195,https://ria.ru/20110203/329932778.html,17:27 03.02.2011,чиновник должный рассказывать свой работа теле...,считать представитель любой властный структура...,москва 3 фев риа новость премьер министр рф вл...
196,https://ria.ru/20110203/329922066.html,16:59 03.02.2011,путин заявлять спокойно относиться критика вра...,слушать некоторый вещь знать самый дело жизнь ...,москва 3 фев риа новость премьер министр рф вл...
197,https://ria.ru/20110203/329921841.html,16:58 03.02.2011,путин говорить дружить свой дочь,премьер признаваться удаваться дружить дочь ра...,москва 3 фев риа новость премьер министр рф вл...


In [5]:
urls = []
with open('/Users/eugenborisenko/Desktop/Универ мага/Диссертация/Парсеры сайтов/ria_economy/raw_preview/news_ria14oct15.html') as file:
    src = file.read()

soup = BeautifulSoup(src, 'lxml')
items_divs = soup.find_all('div', class_='list-item')

for item in items_divs:
    item_url = item.find('div', class_='list-item__content').find('a').get('href')
    if item_url not in urls:
        urls.append(item_url)

In [6]:
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [8]:
start_parsing(urls, n_threads=2, save_evry=200)

0 saved
200 saved
oil_cost_november_01112015
Invalid URL 'oil_cost_november_01112015': No scheme supplied. Perhaps you meant http://oil_cost_november_01112015?
currency_november_01112015
Invalid URL 'currency_november_01112015': No scheme supplied. Perhaps you meant http://currency_november_01112015?
stock_market_november_01112015
Invalid URL 'stock_market_november_01112015': No scheme supplied. Perhaps you meant http://stock_market_november_01112015?
400 saved
600 saved
summit_GECF_23112015
Invalid URL 'summit_GECF_23112015': No scheme supplied. Perhaps you meant http://summit_GECF_23112015?
800 saved
1000 saved
Manila_APEC_summit_17112015
Invalid URL 'Manila_APEC_summit_17112015': No scheme supplied. Perhaps you meant http://Manila_APEC_summit_17112015?
1200 saved
1400 saved
1600 saved
1800 saved
2000 saved
2200 saved
2400 saved
2600 saved
2800 saved
3000 saved
oil_cost_october_01102015
Invalid URL 'oil_cost_october_01102015': No scheme supplied. Perhaps you meant http://oil_cost_oct

### Лента

In [2]:
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

In [15]:
def save_urls(url, urls, user_agent):
    page = 1
    while True:
        time.sleep(1.75)
        url = url + f'page/{page}/'
        data = requests.get(url, headers={'User-Agent':user_agent}).text
        soup = BeautifulSoup(data, 'lxml')
        items = soup.find_all('a', class_='card-full-news _archive')
        if items == []:
            break
        for item in items:
            item_url = 'https://lenta.ru/' + item.get('href')
            if item_url not in urls:
                urls.append(item_url)
        page += 1
    return urls
    


def get_lenta_urls():
    c = 0
    if c % 3 == 0:
        user_agent = user_agent_rotator.get_random_user_agent()
    urls = []
    for year in range(2018, 2023):
        for month in range(1, 13):
            if month in [1, 3, 5, 7, 8, 10, 12]:
                if len(str(month)) == 1:
                    month = '0' + str(month)
                for day in range(1, 32):
                    if len(str(day)) == 1:
                        day = '0' + str(day)
                    url = f'https://lenta.ru/rubrics/economics/{year}/{month}/{day}/'
                    urls = save_urls(url, urls, user_agent)
            elif month in [4, 6, 9, 11]:
                if len(str(month)) == 1:
                    month = '0' + str(month)
                for day in range(1, 31):
                    if len(str(day)) == 1:
                        day = '0' + str(day)
                    url = f'https://lenta.ru/rubrics/economics/{year}/{month}/{day}/'
                    urls = save_urls(url, urls, user_agent)
            else:
                if len(str(month)) == 1:
                    month = '0' + str(month)
                if year % 4 == 0:
                    for day in range(1, 30):
                        if len(str(day)) == 1:
                            day = '0' + str(day)
                        url = f'https://lenta.ru/rubrics/economics/{year}/{month}/{day}/'
                        urls = save_urls(url, urls, user_agent)
                else:
                    for day in range(1, 29):
                        if len(str(day)) == 1:
                            day = '0' + str(day)
                        url = f'https://lenta.ru/rubrics/economics/{year}/{month}/{day}/'
                        urls = save_urls(url, urls, user_agent)
                        
            with open(f'lenta_economy/urls{year}-{month}.txt', 'w') as file:
                for url in urls:
                    file.write(f'{url}\n')
                urls = []
            c += 1          
            print(f'{year}-{month} готов')
    print('Все сохранено')

In [ ]:
get_lenta_urls()

2018-01 готов
2018-02 готов
2018-03 готов
2018-04 готов
2018-05 готов
2018-06 готов


In [ ]:
# with open('ria_politics/urls.txt') as file:
#     urls = file.read()
# urls = urls.split('\n')

In [ ]:
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [5]:
def parse_from_link_lenta(link, user_agent, current_value):
    if current_value % 10 == 0:
        user_agent = user_agent_rotator.get_random_user_agent()

    try:
        with requests.session() as s:
            txt = s.get(link, headers={'User-Agent':user_agent}).text
            soup = BeautifulSoup(txt, 'lxml')
            try:
                timestamp = soup.find('a', class_='topic-header__item topic-header__time').text
            except:
                timestamp = 'No time'
            
            try:
                title = soup.find('span', class_='topic-body__title').text
            except:
                title = 'No title'
            try:
                announce = soup.find('div', class_='topic-body__title-yandex').text
            except:
                announce = 'No announce'
            try:
                text = soup.find_all('p', class_='topic-body__content-text')
                text = ' '.join([el.text for el in text])
            except:
                text = 'No text'
            return timestamp, title, announce, text

    except Exception as ex:
        print(link)
        print(ex)
        return ex

    except KeyboardInterrupt:
        return

In [ ]:
def parse(lock, links, save_evry):

    global data, total_news, current_value

    user_agent = user_agent_rotator.get_random_user_agent()

    while current_value < total_news:
        with lock:
            thread_value = current_value
            link = links[current_value]
            current_value += 1
        
        parsed = parse_from_link_lenta(link, user_agent, thread_value)
        if parsed == None:
            return

        with lock:
            data['link'].append(link)
            data['data_or_ex'].append(parsed)
            if thread_value % save_evry == 0:
                with open(f'Парсеры сайтов/lenta_economy/raw_cites/{thread_value}.pkl', 'wb') as f:
                    pickle.dump(data, f)
                    print(f'{thread_value} saved')
                data.clear()
                data['link'] = []
                data['data_or_ex'] = []

        time.sleep(1.75)

    with open(f'Парсеры сайтов/lenta_economy/raw_cites/{thread_value}.pkl', 'wb') as f:
        pickle.dump(data, f)

    return 'Конец'

def start_parsing(links, n_threads=2, save_evry=15000):
    
    lock = Lock()
    threads = []
    run_event = threading.Event()
    run_event.set()

    for i in range(n_threads):
        t = Thread(target=parse, args=(lock, links, save_evry))
        t.start()
        time.sleep(1.75)
        threads.append(t)

    for thread in threads:
        thread.join()

In [ ]:
start_parsing(urls, save_evry=200)

### РБК

In [1]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [8]:
class rbc_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/v10/search/ajax/?\
        project={0}&\
        category={1}&\
        dateFrom={2}&\
        dateTo={3}&\
        offset={4}&\
        limit={5}&\
        query={6}&\
        material={7}'.format(param_dict['project'],
                            param_dict['category'],
                            param_dict['dateFrom'],
                            param_dict['dateTo'],
                            param_dict['offset'],
                            param_dict['limit'],
                            param_dict['query'],
                            param_dict['material'])
        
        return url
    
    
    def _get_search_table(self, param_dict: dict,
                          includeText: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        
        includeText: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['items'])
        if includeText and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()    
        return search_table
    
    
    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None
        
        return overview, text 
    
    def get_articles(self,
                     param_dict,
                     time_step = 7,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_csv("checkpoint_table.csv")
                print('Checkpoint saved!')
                save_counter = 0
        
        if save_excel:
            out.to_csv("rbc_{}_{}.csv".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')
        
        return out

In [17]:
# Задаем параметры запросы и складываем в param_dict
use_parser = "РБК"

query = 'РБК'
project = ""
category = "economics"
material = ""
dateFrom = '2021-01-01'
dateTo = "2021-02-28"
offset = 0
limit = 100

if use_parser == "РБК":
    param_dict = {'query'   : query, 
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'offset'  : str(offset),
                  'limit'   : str(limit),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': '', 'category': 'economics', 'dateFrom': '01.01.2021', 'dateTo': '28.02.2021', 'offset': '0', 'limit': '5', 'material': ''}


In [18]:
# Пример того, как выглядит json таблица запроса по параметрам.
# Действует ограничение в 100 статей на 1 запрос (параметром limit)
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict,
                               includeText = True) # Парсить текст статей
print(len(tbl))
tbl.head()

10


,id,fronturl,publish_date_t,publish_date,title,photo,project,category,opinion_authors,authors,anons,overview,text
0,63b308ac9a7947e7e48fbbf1,https://www.rbc.ru/rbcfreenews/63b308ac9a7947e...,1672680747,"Mon, 02 Jan 2023 20:32:27 +0300",Военный комитет НАТО обсудит конфликты на Укра...,{'url': ''},None,Политика,None,None,"... «обеспечении большей стабильности, созда...",None,"Высший военный орган НАТО, Военный комитет, пр..."
1,63b308a49a7947e7e48fbbef,https://www.rbc.ru/politics/02/01/2023/63b308a...,1672679469,"Mon, 02 Jan 2023 20:11:09 +0300",Самарский губернатор рассказал о погибших при ...,{'url': 'https://s0.rbk.ru/v6_top_pics/resized...,None,Политика,None,[Владислав Гордеев],... . Семьи военнослужащих могут рассчитывать...,При ракетном ударе по Макеевке погибли в том ч...,Губернатор Самарской области Дмитрий Азаров со...
2,621a39ba9a79472784f029d4,https://www.rbc.ru/politics/02/01/2023/621a39b...,1672678252,"Mon, 02 Jan 2023 19:50:52 +0300",Военная операция на Украине. Карта,{'url': ''},None,Политика,None,[Редакция РБК],... один указ президента устанавливает разны...,Россия начала военную операцию на Украине 24 ф...,2 января Минобороны России сообщило об ударе ...
3,63b2fe8a9a7947e6a2883245,https://www.rbc.ru/rbcfreenews/63b2fe8a9a7947e...,1672677400,"Mon, 02 Jan 2023 19:36:40 +0300",Суд арестовал обвиняемого в убийстве сотрудник...,{'url': ''},None,Общество,None,None,... нападавший 16 раз ударил погибшего ножом...,None,Тверской суд Москвы арестовал обвиняемого в у...
4,63b2ff8b9a7947e649b8b743,https://www.rbc.ru/politics/02/01/2023/63b2ff8...,1672676691,"Mon, 02 Jan 2023 19:24:51 +0300",Принц Гарри заявил о желании вернуть отца и брата,{'url': 'https://s0.rbk.ru/v6_top_pics/resized...,None,Политика,None,[Владислав Гордеев],... не институцию». Полностью интервью будет...,Принц Гарри и его жена не снижают уровень меди...,"Герцог Сассекский принц Гарри считает, что кор..."


In [ ]:
table = parser.get_articles(param_dict=param_dict,
                             time_step = 7, # Шаг - 7 дней, можно больше,
                                            # но есть риск отсечения статей в неделях, гдестатей больше 100
                             save_every = 5, # Сохранять чекпойнт каждые 5 шагов
                             save_excel = True) # Сохранить итоговый файл
print(len(table))
table.head()

### Ведомости

In [2]:
def get_file_urls(file_path):
    import os
    files = sorted(os.listdir(file_path))
    files = [file for file in files if 'html' in file]

    urls = []
    c = 1
    for f in files:

        with open(file_path + f'/{f}', encoding='utf-8') as file:
            src = file.read()
        
        soup = BeautifulSoup(src, 'lxml')
        items_divs = soup.find_all('a', class_="article-preview-item articles-preview-list__item")

        for item in items_divs:
            item_url = item.get('href')
            if item_url not in urls:
                urls.append(item_url)

        print(f'Ссылки из файлов {c} из {len(files)} получены')
        c+=1
        with open(f'vedomosti_invest/urls_{f[10:-5]}.txt', 'w') as file:
            for url in urls:
                file.write(f'{url}\n')
            urls = []

        print(f'Ссылки сохранены')

In [4]:
get_file_urls('vedomosti_business')

Ссылки из файлов 1 из 1 получены
Ссылки сохранены


In [2]:
def parse_from_link_vedomosti(link, user_agent, current_value):
    if current_value % 10 == 0:
        user_agent = user_agent_rotator.get_random_user_agent()

    try:
        with requests.session() as s:
            txt = s.get(link, headers={'User-Agent':user_agent}).text
            soup = BeautifulSoup(txt, 'lxml')
            try:
                timestamp = soup.find('time', class_='article-meta__date').get('datetime')
            except:
                timestamp = 'No time'
            
            try:
                title = soup.find('h1', class_='article-headline__title').text
            except:
                title = 'No title'
            try:
                announce = soup.find('em', class_='article-headline__subtitle').text
            except:
                announce = 'No announce'
            try:
                text = soup.find_all('p', class_='box-paragraph__text')
                text = ' '.join([el.text for el in text])
            except:
                text = 'No text'
            return timestamp, title, announce, text

    except Exception as ex:
        print(link)
        print(ex)
        return ex

    except KeyboardInterrupt:
        return

In [9]:
def parse(lock, links, save_evry):

    global data, total_news, current_value

    user_agent = user_agent_rotator.get_random_user_agent()

    while current_value < total_news:
        with lock:
            thread_value = current_value
            link = links[current_value]
            current_value += 1
        
        parsed = parse_from_link_vedomosti(link, user_agent, thread_value)
        if parsed == None:
            return

        with lock:
            data['link'].append(link)
            data['data_or_ex'].append(parsed)
            if thread_value % save_evry == 0:
                with open(f'vedomosti_invest/raw_cites/{thread_value}.pkl', 'wb') as f:
                    pickle.dump(data, f)
                    print(f'{thread_value} saved')
                data.clear()
                data['link'] = []
                data['data_or_ex'] = []

        time.sleep(1)

    with open(f'vedomosti_invest/raw_cites/{thread_value}.pkl', 'wb') as f:
        pickle.dump(data, f)

    return 'Конец'

def start_parsing(links, n_threads=2, save_evry=15000):
    
    lock = Lock()
    threads = []
    run_event = threading.Event()
    run_event.set()

    for i in range(n_threads):
        t = Thread(target=parse, args=(lock, links, save_evry))
        t.start()
        time.sleep(1)
        threads.append(t)

    for thread in threads:
        thread.join()

In [4]:
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

In [12]:
with open('vedomosti_invest/urls_invest.txt') as file:
    urls = file.read()
urls = urls.split('\n')

In [13]:
data = {'link': [], 'data_or_ex': []}
total_news = len(urls)
current_value = 0

In [14]:
total_news

193

In [15]:
start_parsing(urls, n_threads=2, save_evry=200)

0 saved

Invalid URL '': No scheme supplied. Perhaps you meant http://?
